# Feature Engineering 

- Aditya Rachman Putra -23520032

- Harits Abdurrohman - 23520047

In [1]:
from pycocotools.coco import COCO
from sklearn import tree, metrics
from skimage.transform import rescale, resize, downscale_local_mean
from skimage.util import crop
from skimage import io, color
from math import floor, ceil
from PIL import Image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.decomposition import FastICA
from sklearn.decomposition import TruncatedSVD
import json
from LoadLocalCOCO import LoadLocalCOCO as llc
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
import pickle
from joblib import dump, load
from tqdm import tqdm
import cv2
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, PredefinedSplit
import xgboost as xgb

# GLCM
from skimage.feature import greycomatrix, greycoprops

%matplotlib inline

mau mastiin lagi. Jadi data yang dipakai itu train+val dari coco 2017. Terus nanti pakai train_val_split 2 kali buat dapet val sama test. Baru difed ke gridseachCV

In [3]:
def flatten_list(qlist):
    return [y for x in qlist for y in x]

In [4]:
train_instance="annotations/instances_train2017.json"
val_instance="annotations/instances_val2017.json"

coco_train_path = "D:/FSR/COCO/train2017"
coco_val_path = "D:/FSR/COCO/val2017"

classes = ['dog', 'cat']

coco_train = llc(train_instance, coco_train_path)
coco_val = llc(val_instance, coco_val_path)

loading annotations into memory...
Done (t=21.39s)
creating index...
index created!
loading annotations into memory...
Done (t=0.61s)
creating index...
index created!


In [5]:
data_train = []
y_train = []
for i, category in enumerate(classes):
    data = coco_train.get_cropped_images_flatten_and_resized(category, resize=(128, 128), flatten=False)
    data_train.append(data)
    y_train += np.array(len(data)) *[abs(1-i)]

5508it [06:02, 15.21it/s]
2it [00:00, 14.98it/s]Broken images: [7125, 16775, 24247, 24247, 55296, 47386, 69468, 35613, 77325, 12696, 22427, 26321, 54957, 790, 1360, 55478, 42341, 57631, 2400, 10276, 94049, 58741, 5782, 44613, 58741, 24159, 44613, 29176] images
4768it [05:37, 14.15it/s]Broken images: [5617, 711, 39171, 9413, 46749, 18211, 26375, 26375, 26375, 66041] images



In [6]:
data_val = []
y_val = []
for i, category in enumerate(classes):
    data = coco_val.get_cropped_images_flatten_and_resized(category, resize=(128, 128), flatten=False)
    data_val.append(data)
    y_val += np.array(len(data)) *[abs(1-i)]

218it [00:03, 71.88it/s]
7it [00:00, 69.28it/s]Broken images: [] images
202it [00:02, 75.35it/s]Broken images: [] images



In [8]:
y_train = np.array(len(data_train[0]) *[1] + len(data_train[1]) *[0])
np.shape(y_train)

(10238,)

In [9]:
y_val = np.array(len(data_val[0]) *[1] + len(data_val[1]) *[0])
np.shape(y_val)

(420,)

In [12]:
y = list(y_train) + list(y_val)
np.shape(y)

(10658,)

In [15]:
X = flatten_list(data_train)+flatten_list(data_val)
np.shape(X)

(10658, 128, 128, 3)

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5, random_state=101)

In [19]:
print('data train dimension:', np.shape(X_train))
print('label train dimension:', np.shape(y_train))
print('data val dimension:', np.shape(X_val))
print('label val dimension:', np.shape(y_val))
print('data test dimension:', np.shape(X_test))
print('label test dimension:', np.shape(y_test))

data train dimension: (7460, 128, 128, 3)
label train dimension: (7460,)
data val dimension: (1599, 128, 128, 3)
label val dimension: (1599,)
data test dimension: (1599, 128, 128, 3)
label test dimension: (1599,)


## Ekstraksi fitur

Fitur-fitur yang digunakan pada eksperimen ini:

- **GLCM (Gray level co-occurrence Matrix)**: Merupakan sebuah histogram 2 dimensi yang setiap selnya merupakan jumlah kemunculan kombinasi pixel greyscale. Pada eksperimen ini, distance yang digunakan ada 5 dengan sudut ketetanggaannya 0 derajat. Fitur yang diambil dari matrix ini antara lain adalah contrast, dissimilarity, homogeneity, ASM, energy dan correlation
- **Color Moments (4 moments)**: Merupakan representasi dari distribusi warna pada suatu citra. Momen yang diambil antara lain adalah mean, standard deviation, skewness dan kurtosis
- **Kombinasi (GLCM dan Color Moment)**: Kombinasi dari GLCM dan color moment

In [20]:
# tekstur
# glcm
def glcm_feature_extraction(image_data, distance=[5], angles=[0], levels=256, symmetric=True, normed=True, features=['contrast', 'dissimilarity', 'homogeneity', 'ASM', 'energy', 'correlation']):
    feature_data = []
    for image in image_data:
        patch_feature = {}
        if len(image.shape)==3:
            image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        glcm = greycomatrix(image, distances=distance, angles=angles, levels=levels, symmetric=symmetric, normed=normed)
        for feature in features:
            patch_feature[feature] = greycoprops(glcm, feature)[0, 0]
        feature_data.append(patch_feature)
    feature_data = pd.DataFrame(feature_data)
    return feature_data

In [21]:
glcm_data_train = glcm_feature_extraction(X_train)

In [22]:
np.shape(glcm_data_train)

(7460, 6)

In [23]:
glcm_data_train.head()

,contrast,dissimilarity,homogeneity,ASM,energy,correlation
0,2744.931847,34.522548,0.073339,0.000149,0.012189,0.724956
1,639.753430,15.247459,0.149512,0.000420,0.020486,0.904713
2,3261.264609,36.928989,0.064879,0.000111,0.010536,0.686250
3,1415.773438,19.455729,0.142198,0.000946,0.030762,0.522030
4,3824.776677,44.086890,0.045690,0.000084,0.009157,0.503423


In [24]:
glcm_data_train.to_csv('glcm_data_train_128.csv')

In [25]:
glcm_data_test = glcm_feature_extraction(X_test)
glcm_data_test.head()

,contrast,dissimilarity,homogeneity,ASM,energy,correlation
0,1436.154916,25.480119,0.061661,0.000114,0.010669,0.791768
1,992.162983,20.804116,0.067780,0.000118,0.010873,0.796271
2,1367.497205,20.361153,0.147659,0.000580,0.024079,0.784841
3,1378.903836,21.863186,0.132433,0.000692,0.026310,0.719942
4,735.610963,12.183117,0.210797,0.001473,0.038382,0.916986


In [26]:
np.shape(glcm_data_test)

(1599, 6)

In [27]:
glcm_data_test.to_csv('glcm_data_test_128.csv')

In [28]:
glcm_data_val = glcm_feature_extraction(X_val)
glcm_data_val.head()

,contrast,dissimilarity,homogeneity,ASM,energy,correlation
0,1186.267022,20.462398,0.142423,0.000409,0.020230,0.918714
1,1864.278582,29.090574,0.054027,0.000097,0.009835,0.699873
2,614.920986,17.735518,0.084020,0.000160,0.012663,0.851560
3,2034.752731,29.316883,0.087342,0.000152,0.012326,0.687875
4,1361.513021,25.682736,0.065934,0.000124,0.011127,0.765719


In [29]:
np.shape(glcm_data_val)

(1599, 6)

In [30]:
glcm_data_val.to_csv('glcm_data_val_128.csv')

In [31]:
# color moment
# moment 1: mean
# moment 2: standard deviation
# moment 3: skewness
# moment 4: kurtosis

def modified_skew(image):
    mean = np.average(image)
    dime = np.shape(image)[0]*np.shape(image)[1]
    skew_img = np.power((image.astype(np.int8) - mean), 3).sum()
    skew_img *= 1/dime
    skew_img = np.cbrt(skew_img)
    return skew_img

def modified_skew2(image):
    mean = np.average(image)
    dime = np.shape(image)[0]*np.shape(image)[1]
    std = np.std(image)
    skew_img = np.power(((image.astype(np.int8) - mean)/std), 3)
    skew_img *= 1/dime
    return skew_img

def modified_kurtosis(image):
    mean = np.average(image)
    std = np.std(image)
    dime = np.shape(image)[0]*np.shape(image)[1]
    kurt = (np.power(((image.astype(np.int8) - mean)/std), 4)-3).sum()
    kurt *= 1/dime
    return kurt


def color_moment_extraction_3channel(image_data):
    feature_data = []
    for image in image_data:
        image_features = {}
        for ch in range(image.shape[2]):
            mean_img = np.average(image[:,:,ch])
            image_features[f'mean_{ch}'] = mean_img
            std_img = np.std(image[:,:,ch])
            image_features[f'std_{ch}'] = std_img
            skew_img = modified_skew(image[:,:,ch])
            image_features[f'skewness_{ch}'] = skew_img
            kurtosis_img = modified_kurtosis(image[:,:,ch])
            image_features[f'kurtosis_{ch}'] = kurtosis_img
        feature_data.append(image_features)
    feature_data = pd.DataFrame(feature_data)
    return feature_data

def color_moment_extraction_1channel(image_data):
    feature_data = []
    for image in image_data:
        image_features = {}
        mean_img = np.average(image)
        image_features['mean'] = mean_img
        std_img = np.std(image)
        image_features['std'] = std_img
        skew_img = modified_skew(image)
        image_features['skewness'] = skew_img
        kurtosis_img = modified_kurtosis(image)
        image_features['kurtosis'] = kurtosis_img
        feature_data.append(image_features)
    feature_data = pd.DataFrame(feature_data)
    return feature_data


In [32]:
color_moment_data_train = color_moment_extraction_3channel(X_train)

In [33]:
np.shape(color_moment_data_train)

(7460, 12)

In [34]:
color_moment_data_train.head()

,mean_0,std_0,skewness_0,kurtosis_0,mean_1,std_1,skewness_1,kurtosis_1,mean_2,std_2,skewness_2,kurtosis_2
0,136.172058,72.856767,-168.688403,31.746288,139.016846,72.390883,-174.519154,37.299883,144.231140,74.379487,-179.140860,36.286735
1,89.266785,64.739996,-95.469450,4.365631,91.137085,59.160984,-102.431257,11.106736,87.084717,53.623132,-110.010663,24.921714
2,108.071716,70.524093,-145.372340,20.357592,107.105835,73.210524,-141.211342,15.013502,112.882812,76.420552,-150.872615,16.006257
3,20.065979,33.687491,-19.070275,0.243184,40.243286,39.543853,-40.215852,1.972069,50.113708,43.833341,-52.123000,3.293752
4,78.503296,63.723269,-97.838097,6.499072,112.477905,64.310615,-139.954633,28.869039,139.664124,64.460347,-173.634888,63.930074


In [35]:
color_moment_data_train.to_csv('color_moment_data_train_128.csv')

In [36]:
color_moment_data_test = color_moment_extraction_3channel(X_test)

In [37]:
np.shape(color_moment_data_test)

(1599, 12)

In [38]:
color_moment_data_test.head()

,mean_0,std_0,skewness_0,kurtosis_0,mean_1,std_1,skewness_1,kurtosis_1,mean_2,std_2,skewness_2,kurtosis_2
0,58.534363,45.414771,-66.820328,8.696499,105.589661,63.515600,-152.184384,39.498005,96.280457,57.708556,-140.270431,45.304135
1,122.608215,21.593151,-169.891635,5409.250146,112.304565,50.044327,-152.692020,115.704224,125.010498,61.225492,-167.209221,67.319510
2,67.037598,47.289786,-75.062002,10.919097,83.531555,57.676117,-92.830170,9.361874,86.063538,58.529974,-111.360144,18.842864
3,35.983582,51.929194,-48.494225,-0.563226,42.185669,50.340243,-51.686680,0.219246,45.750488,50.332394,-54.707188,0.718740
4,53.730469,59.091021,-102.646476,11.519377,62.419067,66.825585,-108.449483,6.898668,71.457520,68.426906,-108.675095,5.967658


In [39]:
color_moment_data_test.to_csv('color_moment_data_test_128.csv')

In [40]:
color_moment_data_val = color_moment_extraction_3channel(X_val)
np.shape(color_moment_data_val)

(1599, 12)

In [41]:
color_moment_data_val.head()

,mean_0,std_0,skewness_0,kurtosis_0,mean_1,std_1,skewness_1,kurtosis_1,mean_2,std_2,skewness_2,kurtosis_2
0,129.627075,86.583791,-152.725852,8.950266,126.298767,86.704394,-149.831248,8.127143,129.056152,83.627583,-150.675285,10.310350
1,100.414917,47.591681,-141.832634,110.133103,142.438477,58.938581,-177.252441,98.944605,196.737183,55.790011,-240.789833,374.352238
2,69.503845,35.945690,-64.841775,27.565002,90.246033,45.073452,-115.954801,70.924188,114.482239,53.238812,-156.814806,98.914417
3,81.094177,52.235044,-97.460576,18.512349,85.246826,59.394996,-107.977917,14.590429,96.394775,58.496970,-119.039118,23.489220
4,106.919250,53.578233,-128.580216,47.500135,106.919250,53.578233,-128.580216,47.500135,106.919250,53.578233,-128.580216,47.500135


In [42]:
color_moment_data_val.to_csv('color_moment_data_val_128.csv')

In [43]:
# texture
# HOG
from skimage import feature
from sklearn.decomposition import PCA, IncrementalPCA
from scipy import sparse
def hog_feature_extraction(image_data, orientations=9, pixels_per_cell=(8, 8),
	cells_per_block=(2, 2), transform_sqrt=True, block_norm="L1", n_components=10, threshold=0.45):
    feature_data = []
    hog_features = []
    # pca = PCA(n_components = n_components, svd_solver='full')
    pca = IncrementalPCA(n_components=n_components, batch_size=300)
    for image in image_data:
        hog_f = feature.hog(image, orientations=orientations, pixels_per_cell=pixels_per_cell,
	cells_per_block=cells_per_block, transform_sqrt=transform_sqrt, block_norm=block_norm)
        hog_features.append(hog_f)
    hog_features = np.array(hog_features)
    hog_features[hog_features==np.inf] = 0
    hog_features[hog_features==np.nan] = 0
    print(hog_features.shape)
    # print(np.isnan(np.sum(hog_features)))
    pca.partial_fit(hog_features[:100, :])
    hog_features_sparse = sparse.csr_matrix(hog_features)
    # pca.fit(hog_features_sparse)
    reduced_data_pca = pca.fit_transform(hog_features_sparse)
    for image_pc in reduced_data_pca:
        selected_pc = []
        pc_sum = 0
        for pc in image_pc:
            if pc_sum <=threshold:
                pc_sum += pc
                selected_pc.append(pc)
        features_tmp = {}
        for i, pc in enumerate(selected_pc):
            features_tmp[f'pc_{i}'] = pc
        feature_data.append(features_tmp)    
    feature_data = pd.DataFrame(feature_data)       
    return feature_data

In [320]:
## Bagian GridSearch
# Eksperimen dengan GLCM
test_fold = np.concatenate([
    # The training data.
    np.full(len(glcm_data_train), -1, dtype=np.int8),
    # The development data.
    np.zeros(len(glcm_data_val), dtype=np.int8)
])

cv = PredefinedSplit(test_fold)

X = np.concatenate([glcm_data_train, glcm_data_val])
y = np.concatenate([y_train, y_val])

In [321]:
base = DecisionTreeClassifier()
grid = {
"max_depth" : [10, 100, 1000],
"criterion" : ["gini", "entropy"]
}
model = GridSearchCV(base, grid, cv=cv,n_jobs=4, verbose=100, return_train_score = True)

In [322]:
model = model.fit(X,y)

Fitting 1 folds for each of 6 candidates, totalling 6 fits
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
Pickling array (shape=(9059, 6), dtype=float64).
Pickling array (shape=(9059,), dtype=int32).
Pickling array (shape=(7460,), dtype=int32).
Pickling array (shape=(1599,), dtype=int32).
Pickling array (shape=(9059, 6), dtype=float64).
Pickling array (shape=(9059,), dtype=int32).
Pickling array (shape=(7460,), dtype=int32).
Pickling array (shape=(1599,), dtype=int32).
Pickling array (shape=(9059, 6), dtype=float64).
Pickling array (shape=(9059,), dtype=int32).
Pickling array (shape=(7460,), dtype=int32).
Pickling array (shape=(1599,), dtype=int32).
Pickling array (shape=(9059, 6), dtype=float64).
Pickling array (shape=(9059,), dtype=int32).
Pickling array (shape=(7460,), dtype=int32).
Pickling array (shape=(1599,), dtype=int32).
Pickling array (shape=(9059, 6), dtype=float64).
Pickling array (shape=(9059,), dtype=int32).
Pickling array (shape=(7460,), dtype

In [323]:
# print(model.cv_results_)

In [324]:
df = pd.concat([pd.DataFrame(model.cv_results_["rank_test_score"], columns=["Rank"]), pd.DataFrame(model.cv_results_["params"]),pd.DataFrame(model.cv_results_["mean_test_score"], columns=["Accuracy"]),pd.DataFrame(model.cv_results_["mean_train_score"], columns=["Train Score"])],axis=1)
df.to_csv('local_dtl_accuracy_{}_glcm.csv'.format(128))
df

,Rank,criterion,max_depth,Accuracy,Train Score
0,1,gini,10,0.556598,0.673861
1,5,gini,100,0.534709,1.000000
2,3,gini,1000,0.539712,1.000000
3,2,entropy,10,0.555347,0.649732
4,4,entropy,100,0.535335,1.000000
5,6,entropy,1000,0.530331,1.000000


In [325]:
df.loc[df['Accuracy'].idxmax()]

Rank                  1
criterion          gini
max_depth            10
Accuracy       0.556598
Train Score    0.673861
Name: 0, dtype: object

In [326]:
predictions = model.predict(glcm_data_test)
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       0.52      0.36      0.42       723
           1       0.58      0.73      0.64       876

    accuracy                           0.56      1599
   macro avg       0.55      0.54      0.53      1599
weighted avg       0.55      0.56      0.54      1599



In [327]:
dump(model, 'local_dtl_{}_{}_glcm.pkl'.format('70p', 128))

['local_dtl_70p_128_glcm.pkl']

In [328]:
# Random Forest classifier
rf_base = xgb.XGBRFClassifier()
rf_grid = {
"max_depth" : [4, 6, 10],
"learning_rate" : [0.1, 0.3],
"n_estimators" : [1, 5, 10, 50]
}
rf_model = GridSearchCV(rf_base, rf_grid, cv=cv,n_jobs=4, verbose=100, return_train_score = True)

In [329]:
rf_model = rf_model.fit(X, y)

Fitting 1 folds for each of 24 candidates, totalling 24 fits
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
Pickling array (shape=(9059, 6), dtype=float64).
Pickling array (shape=(9059,), dtype=int32).
Pickling array (shape=(7460,), dtype=int32).
Pickling array (shape=(1599,), dtype=int32).
Pickling array (shape=(9059, 6), dtype=float64).
Pickling array (shape=(9059,), dtype=int32).
Pickling array (shape=(7460,), dtype=int32).
Pickling array (shape=(1599,), dtype=int32).
Pickling array (shape=(9059, 6), dtype=float64).
Pickling array (shape=(9059,), dtype=int32).
Pickling array (shape=(7460,), dtype=int32).
Pickling array (shape=(1599,), dtype=int32).
Pickling array (shape=(9059, 6), dtype=float64).
Pickling array (shape=(9059,), dtype=int32).
Pickling array (shape=(7460,), dtype=int32).
Pickling array (shape=(1599,), dtype=int32).
Pickling array (shape=(9059, 6), dtype=float64).
Pickling array (shape=(9059,), dtype=int32).
Pickling array (shape=(7460,), dty

In [330]:
# print(rf_model.cv_results_)

In [331]:
df = pd.concat([pd.DataFrame(rf_model.cv_results_["rank_test_score"], columns=["Rank"]), pd.DataFrame(rf_model.cv_results_["params"]),pd.DataFrame(rf_model.cv_results_["mean_test_score"], columns=["Accuracy"]),pd.DataFrame(rf_model.cv_results_["mean_train_score"], columns=["Train Score"])],axis=1)
df.to_csv('local_rf_accuracy_{}_glcm.csv'.format(128))
df

,Rank,learning_rate,max_depth,n_estimators,Accuracy,Train Score
0,9,0.1,4,1,0.569731,0.568499
1,13,0.1,4,5,0.566604,0.589678
2,5,0.1,4,10,0.573483,0.589678
3,3,0.1,4,50,0.577236,0.594102
4,17,0.1,6,1,0.558474,0.591421
5,21,0.1,6,5,0.557849,0.618901
6,11,0.1,6,10,0.567230,0.633914
7,1,0.1,6,50,0.580363,0.632976
8,23,0.1,10,1,0.544715,0.644638
9,7,0.1,10,5,0.572233,0.723727


In [332]:
df.loc[df['Accuracy'].idxmax()]

Rank              1.000000
learning_rate     0.100000
max_depth         6.000000
n_estimators     50.000000
Accuracy          0.580363
Train Score       0.632976
Name: 7, dtype: float64

In [333]:
dump(rf_model, 'local_rf_{}_{}_glcm.pkl'.format('70p', 128))

['local_rf_70p_128_glcm.pkl']

In [334]:
df_test = glcm_data_test.rename(columns={'contrast': 'f0', 'dissimilarity':'f1', 'homogeneity':'f2', 'ASM':'f3', 'energy':'f4', 'correlation':'f5'})
predictions = rf_model.predict(df_test)
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       0.54      0.45      0.49       723
           1       0.60      0.68      0.64       876

    accuracy                           0.58      1599
   macro avg       0.57      0.57      0.57      1599
weighted avg       0.57      0.58      0.57      1599



In [335]:
xg_base = xgb.XGBClassifier()
xg_grid = {
"max_depth" : [4, 6, 10],
"learning_rate" : [0.1, 0.3],
"n_estimators" : [1, 5, 10, 50]
}
xg_model = GridSearchCV(xg_base, xg_grid, cv=cv, n_jobs=4, verbose=100, return_train_score = True)

In [336]:
xg_model = xg_model.fit(X, y)

Fitting 1 folds for each of 24 candidates, totalling 24 fits
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
Pickling array (shape=(9059, 6), dtype=float64).
Pickling array (shape=(9059,), dtype=int32).
Pickling array (shape=(7460,), dtype=int32).
Pickling array (shape=(1599,), dtype=int32).
Pickling array (shape=(9059, 6), dtype=float64).
Pickling array (shape=(9059,), dtype=int32).
Pickling array (shape=(7460,), dtype=int32).
Pickling array (shape=(1599,), dtype=int32).
Pickling array (shape=(9059, 6), dtype=float64).
Pickling array (shape=(9059,), dtype=int32).
Pickling array (shape=(7460,), dtype=int32).
Pickling array (shape=(1599,), dtype=int32).
Pickling array (shape=(9059, 6), dtype=float64).
Pickling array (shape=(9059,), dtype=int32).
Pickling array (shape=(7460,), dtype=int32).
Pickling array (shape=(1599,), dtype=int32).
Pickling array (shape=(9059, 6), dtype=float64).
Pickling array (shape=(9059,), dtype=int32).
Pickling array (shape=(7460,), dty

In [337]:
df = pd.concat([pd.DataFrame(xg_model.cv_results_["rank_test_score"], columns=["Rank"]), pd.DataFrame(xg_model.cv_results_["params"]),pd.DataFrame(xg_model.cv_results_["mean_test_score"], columns=["Accuracy"]),pd.DataFrame(xg_model.cv_results_["mean_train_score"], columns=["Train Score"])],axis=1)
df.to_csv('local_xg_accuracy_{}_glcm.csv'.format(128))
df

,Rank,learning_rate,max_depth,n_estimators,Accuracy,Train Score
0,14,0.1,4,1,0.547842,0.572922
1,6,0.1,4,5,0.564103,0.600536
2,1,0.1,4,10,0.578487,0.605898
3,3,0.1,4,50,0.567855,0.633780
4,16,0.1,6,1,0.547217,0.605630
5,10,0.1,6,5,0.559725,0.630429
6,11,0.1,6,10,0.552846,0.653083
7,7,0.1,6,50,0.562226,0.700268
8,19,0.1,10,1,0.542214,0.682440
9,13,0.1,10,5,0.548468,0.744772


In [338]:
df.loc[df['Accuracy'].idxmax()]

Rank              1.000000
learning_rate     0.100000
max_depth         4.000000
n_estimators     10.000000
Accuracy          0.578487
Train Score       0.605898
Name: 2, dtype: float64

In [339]:
dump(xg_model, 'local_xg_{}_{}_glcm.pkl'.format('70p', 128))

['local_xg_70p_128_glcm.pkl']

In [340]:
df_test = glcm_data_test.rename(columns={'contrast': 'f0', 'dissimilarity':'f1', 'homogeneity':'f2', 'ASM':'f3', 'energy':'f4', 'correlation':'f5'})
predictions = xg_model.predict(df_test)
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       0.55      0.44      0.49       723
           1       0.60      0.70      0.65       876

    accuracy                           0.58      1599
   macro avg       0.58      0.57      0.57      1599
weighted avg       0.58      0.58      0.58      1599



In [341]:
# Eksperimen dengan Color moment
test_fold = np.concatenate([
    # The training data.
    np.full(len(color_moment_data_train), -1, dtype=np.int8),
    # The development data.
    np.zeros(len(color_moment_data_val), dtype=np.int8)
])

cv = PredefinedSplit(test_fold)

X = np.concatenate([color_moment_data_train, color_moment_data_val])
y = np.concatenate([y_train, y_val])

In [342]:
base = DecisionTreeClassifier()
grid = {
"max_depth" : [10, 100, 1000],
"criterion" : ["gini", "entropy"]
}
model = GridSearchCV(base, grid, cv=cv,n_jobs=4, verbose=100, return_train_score = True)

In [343]:
dtl_X = np.nan_to_num(X)
model = model.fit(dtl_X.astype(np.int8),y)

Fitting 1 folds for each of 6 candidates, totalling 6 fits
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
Pickling array (shape=(9059, 12), dtype=int8).
Pickling array (shape=(9059,), dtype=int32).
Pickling array (shape=(7460,), dtype=int32).
Pickling array (shape=(1599,), dtype=int32).
Pickling array (shape=(9059, 12), dtype=int8).
Pickling array (shape=(9059,), dtype=int32).
Pickling array (shape=(7460,), dtype=int32).
Pickling array (shape=(1599,), dtype=int32).
Pickling array (shape=(9059, 12), dtype=int8).
Pickling array (shape=(9059,), dtype=int32).
Pickling array (shape=(7460,), dtype=int32).
Pickling array (shape=(1599,), dtype=int32).
Pickling array (shape=(9059, 12), dtype=int8).
Pickling array (shape=(9059,), dtype=int32).
Pickling array (shape=(7460,), dtype=int32).
Pickling array (shape=(1599,), dtype=int32).
Pickling array (shape=(9059, 12), dtype=int8).
Pickling array (shape=(9059,), dtype=int32).
Pickling array (shape=(7460,), dtype=int32).
P

In [344]:
df = pd.concat([pd.DataFrame(model.cv_results_["rank_test_score"], columns=["Rank"]), pd.DataFrame(model.cv_results_["params"]),pd.DataFrame(model.cv_results_["mean_test_score"], columns=["Accuracy"]),pd.DataFrame(model.cv_results_["mean_train_score"], columns=["Train Score"])],axis=1)
df.to_csv('local_dtl_accuracy_{}_cm.csv'.format(128))
df

,Rank,criterion,max_depth,Accuracy,Train Score
0,3,gini,10,0.524703,0.687534
1,4,gini,100,0.523452,1.000000
2,5,gini,1000,0.522827,1.000000
3,2,entropy,10,0.526579,0.642225
4,1,entropy,100,0.532833,1.000000
5,6,entropy,1000,0.515322,1.000000


In [345]:
df.loc[df['Accuracy'].idxmax()]

Rank                  1
criterion       entropy
max_depth           100
Accuracy       0.532833
Train Score           1
Name: 4, dtype: object

In [346]:
dump(model, 'local_dtl_{}_{}_cm.pkl'.format('70p', 128))

['local_dtl_70p_128_cm.pkl']

In [347]:
predictions = model.predict(color_moment_data_test)
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       0.47      0.50      0.49       723
           1       0.57      0.54      0.55       876

    accuracy                           0.52      1599
   macro avg       0.52      0.52      0.52      1599
weighted avg       0.52      0.52      0.52      1599



In [348]:
# Random Forest classifier
rf_base = xgb.XGBRFClassifier()
rf_grid = {
"max_depth" : [4, 6, 10],
"learning_rate" : [0.1, 0.3],
"n_estimators" : [1, 5, 10, 50]
}
rf_model = GridSearchCV(rf_base, rf_grid, cv=cv,n_jobs=4, verbose=100, return_train_score = True)

In [349]:
rf_model = rf_model.fit(X, y)

Fitting 1 folds for each of 24 candidates, totalling 24 fits
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
Pickling array (shape=(9059, 12), dtype=float64).
Pickling array (shape=(9059,), dtype=int32).
Pickling array (shape=(7460,), dtype=int32).
Pickling array (shape=(1599,), dtype=int32).
Pickling array (shape=(9059, 12), dtype=float64).
Pickling array (shape=(9059,), dtype=int32).
Pickling array (shape=(7460,), dtype=int32).
Pickling array (shape=(1599,), dtype=int32).
Pickling array (shape=(9059, 12), dtype=float64).
Pickling array (shape=(9059,), dtype=int32).
Pickling array (shape=(7460,), dtype=int32).
Pickling array (shape=(1599,), dtype=int32).
Pickling array (shape=(9059, 12), dtype=float64).
Pickling array (shape=(9059,), dtype=int32).
Pickling array (shape=(7460,), dtype=int32).
Pickling array (shape=(1599,), dtype=int32).
Pickling array (shape=(9059, 12), dtype=float64).
Pickling array (shape=(9059,), dtype=int32).
Pickling array (shape=(7460,)

In [350]:
df = pd.concat([pd.DataFrame(rf_model.cv_results_["rank_test_score"], columns=["Rank"]), pd.DataFrame(rf_model.cv_results_["params"]),pd.DataFrame(rf_model.cv_results_["mean_test_score"], columns=["Accuracy"]),pd.DataFrame(rf_model.cv_results_["mean_train_score"], columns=["Train Score"])],axis=1)
df.to_csv('local_rf_accuracy_{}_cm.csv'.format(128))
df

,Rank,learning_rate,max_depth,n_estimators,Accuracy,Train Score
0,15,0.1,4,1,0.551595,0.579357
1,3,0.1,4,5,0.564103,0.587265
2,5,0.1,4,10,0.561601,0.596917
3,5,0.1,4,50,0.561601,0.593566
4,17,0.1,6,1,0.548468,0.605898
5,9,0.1,6,5,0.559725,0.636059
6,11,0.1,6,10,0.557849,0.651340
7,1,0.1,6,50,0.565353,0.651072
8,21,0.1,10,1,0.542214,0.663673
9,24,0.1,10,5,0.539087,0.749330


In [351]:
df.loc[df['Accuracy'].idxmax()]

Rank              1.000000
learning_rate     0.100000
max_depth         6.000000
n_estimators     50.000000
Accuracy          0.565353
Train Score       0.651072
Name: 7, dtype: float64

In [352]:
dump(rf_model, 'local_rf_{}_{}_cm.pkl'.format('70p', 128))

['local_rf_70p_128_cm.pkl']

In [353]:
df_test = color_moment_data_test.rename(columns={'mean_0':'f0', 'std_0':'f1', 'skewness_0':'f2', 'kurtosis_0':'f3', 'mean_1':'f4', 'std_1':'f5', 'skewness_1':'f6', 'kurtosis_1':'f7', 'mean_2':'f8', 'std_2':'f9', 'skewness_2':'f10', 'kurtosis_2':'f11'})
predictions = rf_model.predict(df_test)
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       0.58      0.42      0.49       723
           1       0.61      0.75      0.67       876

    accuracy                           0.60      1599
   macro avg       0.59      0.58      0.58      1599
weighted avg       0.60      0.60      0.59      1599



In [354]:
xg_base = xgb.XGBClassifier()
xg_grid = {
"max_depth" : [4, 6, 10],
"learning_rate" : [0.1, 0.3],
"n_estimators" : [1, 5, 10, 50]
}
xg_model = GridSearchCV(xg_base, xg_grid, cv=cv, n_jobs=4, verbose=100, return_train_score = True)

In [355]:
xg_model = xg_model.fit(X, y)

Fitting 1 folds for each of 24 candidates, totalling 24 fits
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
Pickling array (shape=(9059, 12), dtype=float64).
Pickling array (shape=(9059,), dtype=int32).
Pickling array (shape=(7460,), dtype=int32).
Pickling array (shape=(1599,), dtype=int32).
Pickling array (shape=(9059, 12), dtype=float64).
Pickling array (shape=(9059,), dtype=int32).
Pickling array (shape=(7460,), dtype=int32).
Pickling array (shape=(1599,), dtype=int32).
Pickling array (shape=(9059, 12), dtype=float64).
Pickling array (shape=(9059,), dtype=int32).
Pickling array (shape=(7460,), dtype=int32).
Pickling array (shape=(1599,), dtype=int32).
Pickling array (shape=(9059, 12), dtype=float64).
Pickling array (shape=(9059,), dtype=int32).
Pickling array (shape=(7460,), dtype=int32).
Pickling array (shape=(1599,), dtype=int32).
Pickling array (shape=(9059, 12), dtype=float64).
Pickling array (shape=(9059,), dtype=int32).
Pickling array (shape=(7460,)

In [356]:
df = pd.concat([pd.DataFrame(xg_model.cv_results_["rank_test_score"], columns=["Rank"]), pd.DataFrame(xg_model.cv_results_["params"]),pd.DataFrame(xg_model.cv_results_["mean_test_score"], columns=["Accuracy"]),pd.DataFrame(xg_model.cv_results_["mean_train_score"], columns=["Train Score"])],axis=1)
df.to_csv('local_xg_accuracy_{}_cm.csv'.format(128))
df

,Rank,learning_rate,max_depth,n_estimators,Accuracy,Train Score
0,12,0.1,4,1,0.555972,0.585791
1,17,0.1,4,5,0.553471,0.592493
2,10,0.1,4,10,0.559099,0.598123
3,7,0.1,4,50,0.559725,0.640080
4,7,0.1,6,1,0.559725,0.606032
5,20,0.1,6,5,0.550969,0.642761
6,15,0.1,6,10,0.555347,0.658311
7,5,0.1,6,50,0.560350,0.735389
8,23,0.1,10,1,0.529081,0.678418
9,18,0.1,10,5,0.552846,0.784853


In [357]:
df.loc[df['Accuracy'].idxmax()]

Rank              1.000000
learning_rate     0.300000
max_depth         6.000000
n_estimators     10.000000
Accuracy          0.567230
Train Score       0.695174
Name: 18, dtype: float64

In [358]:
dump(xg_model, 'local_xg_{}_{}_cm.pkl'.format('70p', 128))

['local_xg_70p_128_cm.pkl']

In [359]:
df_test = color_moment_data_test.rename(columns={'mean_0':'f0', 'std_0':'f1', 'skewness_0':'f2', 'kurtosis_0':'f3', 'mean_1':'f4', 'std_1':'f5', 'skewness_1':'f6', 'kurtosis_1':'f7', 'mean_2':'f8', 'std_2':'f9', 'skewness_2':'f10', 'kurtosis_2':'f11'})
predictions = xg_model.predict(df_test)
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       0.54      0.43      0.48       723
           1       0.60      0.70      0.64       876

    accuracy                           0.58      1599
   macro avg       0.57      0.56      0.56      1599
weighted avg       0.57      0.58      0.57      1599



In [360]:
all_features_train = pd.concat([glcm_data_train, color_moment_data_train],  axis=1, sort=False)
all_features_test = pd.concat([glcm_data_test, color_moment_data_test],  axis=1, sort=False)
all_features_val = pd.concat([glcm_data_val, color_moment_data_val],  axis=1, sort=False)

In [361]:
# Eksperimen dengan seluruh data
test_fold = np.concatenate([
    # The training data.
    np.full(len(all_features_train), -1, dtype=np.int8),
    # The development data.
    np.zeros(len(all_features_val), dtype=np.int8)
])

cv = PredefinedSplit(test_fold)

X = np.concatenate([all_features_train, all_features_val])
y = np.concatenate([y_train, y_val])

In [362]:
base = DecisionTreeClassifier()
grid = {
"max_depth" : [10, 100, 1000],
"criterion" : ["gini", "entropy"]
}
model = GridSearchCV(base, grid, cv=cv,n_jobs=4, verbose=100, return_train_score = True)

In [363]:
dtl_X = np.nan_to_num(X)
model = model.fit(dtl_X.astype(np.int8),y)

Fitting 1 folds for each of 6 candidates, totalling 6 fits
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
Pickling array (shape=(9059, 18), dtype=int8).
Pickling array (shape=(9059,), dtype=int32).
Pickling array (shape=(7460,), dtype=int32).
Pickling array (shape=(1599,), dtype=int32).
Pickling array (shape=(9059, 18), dtype=int8).
Pickling array (shape=(9059,), dtype=int32).
Pickling array (shape=(7460,), dtype=int32).
Pickling array (shape=(1599,), dtype=int32).
Pickling array (shape=(9059, 18), dtype=int8).
Pickling array (shape=(9059,), dtype=int32).
Pickling array (shape=(7460,), dtype=int32).
Pickling array (shape=(1599,), dtype=int32).
Pickling array (shape=(9059, 18), dtype=int8).
Pickling array (shape=(9059,), dtype=int32).
Pickling array (shape=(7460,), dtype=int32).
Pickling array (shape=(1599,), dtype=int32).
Pickling array (shape=(9059, 18), dtype=int8).
Pickling array (shape=(9059,), dtype=int32).
Pickling array (shape=(7460,), dtype=int32).
P

In [364]:
df = pd.concat([pd.DataFrame(model.cv_results_["rank_test_score"], columns=["Rank"]), pd.DataFrame(model.cv_results_["params"]),pd.DataFrame(model.cv_results_["mean_test_score"], columns=["Accuracy"]),pd.DataFrame(model.cv_results_["mean_train_score"], columns=["Train Score"])],axis=1)
df.to_csv('local_dtl_accuracy_{}_all.csv'.format(128))
df

,Rank,criterion,max_depth,Accuracy,Train Score
0,1,gini,10,0.552220,0.704960
1,5,gini,100,0.522827,1.000000
2,3,gini,1000,0.532208,1.000000
3,2,entropy,10,0.542839,0.665818
4,6,entropy,100,0.510944,1.000000
5,4,entropy,1000,0.526579,1.000000


In [365]:
df.loc[df['Accuracy'].idxmax()]

Rank                 1
criterion         gini
max_depth           10
Accuracy       0.55222
Train Score    0.70496
Name: 0, dtype: object

In [366]:
dump(model, 'local_dtl_{}_{}_all.pkl'.format('70p', 128))

['local_dtl_70p_128_all.pkl']

In [367]:
predictions = model.predict(all_features_test)
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       0.50      0.43      0.46       723
           1       0.58      0.64      0.61       876

    accuracy                           0.55      1599
   macro avg       0.54      0.54      0.54      1599
weighted avg       0.54      0.55      0.54      1599



In [368]:
# Random Forest classifier
rf_base = xgb.XGBRFClassifier()
rf_grid = {
"max_depth" : [4, 6, 10],
"learning_rate" : [0.1, 0.3],
"n_estimators" : [1, 5, 10, 50]
}
rf_model = GridSearchCV(rf_base, rf_grid, cv=cv,n_jobs=4, verbose=100, return_train_score = True)

In [369]:
rf_model = rf_model.fit(X, y)

Fitting 1 folds for each of 24 candidates, totalling 24 fits
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
Memmapping (shape=(9059, 18), dtype=float64) to new file C:\Users\glaci\AppData\Local\Temp\joblib_memmapping_folder_27236_9851106777\27236-2182575803464-6db6fb9b6c10411582955da3298d7c5d.pkl
Pickling array (shape=(9059,), dtype=int32).
Pickling array (shape=(7460,), dtype=int32).
Pickling array (shape=(1599,), dtype=int32).
Memmapping (shape=(9059, 18), dtype=float64) to old file C:\Users\glaci\AppData\Local\Temp\joblib_memmapping_folder_27236_9851106777\27236-2182575803464-6db6fb9b6c10411582955da3298d7c5d.pkl
Pickling array (shape=(9059,), dtype=int32).
Pickling array (shape=(7460,), dtype=int32).
Pickling array (shape=(1599,), dtype=int32).
Memmapping (shape=(9059, 18), dtype=float64) to old file C:\Users\glaci\AppData\Local\Temp\joblib_memmapping_folder_27236_9851106777\27236-2182575803464-6db6fb9b6c10411582955da3298d7c5d.pkl
Pickling array (shape=(9

In [370]:
df = pd.concat([pd.DataFrame(rf_model.cv_results_["rank_test_score"], columns=["Rank"]), pd.DataFrame(rf_model.cv_results_["params"]),pd.DataFrame(rf_model.cv_results_["mean_test_score"], columns=["Accuracy"]),pd.DataFrame(rf_model.cv_results_["mean_train_score"], columns=["Train Score"])],axis=1)
df.to_csv('local_rf_accuracy_{}_all.csv'.format(128))
df

,Rank,learning_rate,max_depth,n_estimators,Accuracy,Train Score
0,13,0.1,4,1,0.585991,0.594772
1,9,0.1,4,5,0.586617,0.606568
2,5,0.1,4,10,0.590369,0.609786
3,7,0.1,4,50,0.587867,0.608445
4,17,0.1,6,1,0.581614,0.628418
5,3,0.1,6,5,0.592871,0.657909
6,13,0.1,6,10,0.585991,0.656568
7,1,0.1,6,50,0.593496,0.656702
8,23,0.1,10,1,0.558474,0.685791
9,21,0.1,10,5,0.563477,0.785925


In [371]:
df.loc[df['Accuracy'].idxmax()]

Rank              1.000000
learning_rate     0.100000
max_depth         6.000000
n_estimators     50.000000
Accuracy          0.593496
Train Score       0.656702
Name: 7, dtype: float64

In [372]:
dump(rf_model, 'local_rf_{}_{}_all.pkl'.format('70p', 128))

['local_rf_70p_128_all.pkl']

In [373]:
df_test = all_features_test.rename(columns={'contrast':'f0', 'dissimilarity':'f1', 'homogeneity':'f2', 'ASM':'f3', 'energy':'f4', 'correlation':'f5', 'mean_0':'f6', 'std_0':'f7', 'skewness_0':'f8', 'kurtosis_0':'f9', 'mean_1':'f10', 'std_1':'f11', 'skewness_1':'f12', 'kurtosis_1':'f13', 'mean_2':'f14', 'std_2':'f15', 'skewness_2':'f16', 'kurtosis_2':'f17'})
predictions = rf_model.predict(df_test)
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       0.58      0.51      0.54       723
           1       0.63      0.70      0.67       876

    accuracy                           0.61      1599
   macro avg       0.61      0.60      0.60      1599
weighted avg       0.61      0.61      0.61      1599



In [374]:
xg_base = xgb.XGBClassifier()
xg_grid = {
"max_depth" : [4, 6, 10],
"learning_rate" : [0.1, 0.3],
"n_estimators" : [1, 5, 10, 50]
}
xg_model = GridSearchCV(xg_base, xg_grid, cv=cv, n_jobs=4, verbose=100, return_train_score = True)

In [375]:
xg_model = xg_model.fit(X, y)

Fitting 1 folds for each of 24 candidates, totalling 24 fits
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
Memmapping (shape=(9059, 18), dtype=float64) to new file C:\Users\glaci\AppData\Local\Temp\joblib_memmapping_folder_27236_9851106777\27236-2182575803464-6db6fb9b6c10411582955da3298d7c5d.pkl
Pickling array (shape=(9059,), dtype=int32).
Pickling array (shape=(7460,), dtype=int32).
Pickling array (shape=(1599,), dtype=int32).
Memmapping (shape=(9059, 18), dtype=float64) to old file C:\Users\glaci\AppData\Local\Temp\joblib_memmapping_folder_27236_9851106777\27236-2182575803464-6db6fb9b6c10411582955da3298d7c5d.pkl
Pickling array (shape=(9059,), dtype=int32).
Pickling array (shape=(7460,), dtype=int32).
Pickling array (shape=(1599,), dtype=int32).
Memmapping (shape=(9059, 18), dtype=float64) to old file C:\Users\glaci\AppData\Local\Temp\joblib_memmapping_folder_27236_9851106777\27236-2182575803464-6db6fb9b6c10411582955da3298d7c5d.pkl
Pickling array (shape=(9

In [376]:
df = pd.concat([pd.DataFrame(xg_model.cv_results_["rank_test_score"], columns=["Rank"]), pd.DataFrame(xg_model.cv_results_["params"]),pd.DataFrame(xg_model.cv_results_["mean_test_score"], columns=["Accuracy"]),pd.DataFrame(xg_model.cv_results_["mean_train_score"], columns=["Train Score"])],axis=1)
df.to_csv('local_xg_accuracy_{}_all.csv'.format(128))
df

,Rank,learning_rate,max_depth,n_estimators,Accuracy,Train Score
0,17,0.1,4,1,0.568480,0.599464
1,3,0.1,4,5,0.595372,0.609249
2,9,0.1,4,10,0.582239,0.621582
3,1,0.1,4,50,0.596623,0.671582
4,10,0.1,6,1,0.578487,0.628954
5,5,0.1,6,5,0.585366,0.655496
6,10,0.1,6,10,0.578487,0.679357
7,4,0.1,6,50,0.588493,0.769035
8,19,0.1,10,1,0.567230,0.716622
9,15,0.1,10,5,0.572858,0.822922


In [377]:
df.loc[df['Accuracy'].idxmax()]

Rank              1.000000
learning_rate     0.100000
max_depth         4.000000
n_estimators     50.000000
Accuracy          0.596623
Train Score       0.671582
Name: 3, dtype: float64

In [378]:
dump(xg_model, 'local_xg_{}_{}_all.pkl'.format('70p', 128))

['local_xg_70p_128_all.pkl']

In [379]:
df_test = all_features_test.rename(columns={'contrast':'f0', 'dissimilarity':'f1', 'homogeneity':'f2', 'ASM':'f3', 'energy':'f4', 'correlation':'f5', 'mean_0':'f6', 'std_0':'f7', 'skewness_0':'f8', 'kurtosis_0':'f9', 'mean_1':'f10', 'std_1':'f11', 'skewness_1':'f12', 'kurtosis_1':'f13', 'mean_2':'f14', 'std_2':'f15', 'skewness_2':'f16', 'kurtosis_2':'f17'})
predictions = xg_model.predict(df_test)
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       0.60      0.49      0.54       723
           1       0.63      0.72      0.68       876

    accuracy                           0.62      1599
   macro avg       0.61      0.61      0.61      1599
weighted avg       0.62      0.62      0.61      1599



More on: https://github.com/otakbeku/modified-FCOS